In [1]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Fri May  5 22:47:01 2017

@author: zhouyi
"""

import glob
import os
import sys

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.sparse
import pandas as pd


import confidence_expomf
import evaluation

DATA_ROOT = '/Users/zhouyi/Documents/GraduatePJ/MSD'

In [2]:
unique_uid = list()
with open(os.path.join(DATA_ROOT, 'unique_uid_sub.txt'), 'r') as f:
    for line in f:
        unique_uid.append(line.strip())
        
unique_sid = list()
with open(os.path.join(DATA_ROOT, 'unique_sid_sub.txt'), 'r') as f:
    for line in f:
        unique_sid.append(line.strip())

n_songs = len(unique_sid)
n_users = len(unique_uid)

In [3]:
#Load the data and train the model
def load_data(csv_file, shape=(n_users, n_songs)):
    tp = pd.read_csv(csv_file)
    rows, cols = np.array(tp['uid'], dtype=np.int32), np.array(tp['sid'], dtype=np.int32)
    count = tp['count']
    return scipy.sparse.csr_matrix((count,(rows, cols)), dtype=np.int16, shape=shape), rows, cols

train_data, rows, cols = load_data(os.path.join(DATA_ROOT, 'train.num.sub.csv'))

raw_train_data=train_data
## binarize the data
#train_data.data = np.ones_like(train_data.data)
print 'train_data.shape:',train_data.shape
print 'train_data entries:',train_data.data.shape

vad_data, rows_vad, cols_vad = load_data(os.path.join(DATA_ROOT, 'vad.num.sub.csv'))
## binarize the data
vad_data.data = np.ones_like(vad_data.data)
print 'vad__data.shape:',vad_data.shape
print 'vad_data entries:',vad_data.data.shape
test_data, rows_test, cols_test = load_data(os.path.join(DATA_ROOT, 'test.num.sub.csv'))
## binarize the data
test_data.data = np.ones_like(test_data.data)
print 'test_data.shape:',test_data.shape
print 'test_data entries:',test_data.data.shape
print 'sparse according to vad_data:',float(vad_data.data.shape[0])/(vad_data.shape[0]*vad_data.shape[1])
print 'sparse according to all',float(vad_data.data.shape[0]+raw_train_data.data.shape[0]+test_data.data.shape[0])/(vad_data.shape[0]*vad_data.shape[1])


train_data.shape: (1013, 400)
train_data entries: (24587,)
vad__data.shape: (1013, 400)
vad_data entries: (2731,)
test_data.shape: (1013, 400)
test_data entries: (6829,)
sparse according to vad_data: 0.00673988153998
sparse according to all 0.084271964462


In [4]:
def myEvaluation(model,X,vad_data):

    vad_recall_at_k = evaluation.Recall_at_k(X, vad_data,
                                                model.theta,
                                                model.beta,
                                                k=20)
        
    vad_map_at_k = evaluation.MAP_at_k(X, vad_data,
                                                model.theta,
                                                model.beta,
                                                k=20)
        
    vad_ndcg_at_k = evaluation.NDCG_at_k(X, vad_data,
                                                model.theta,
                                                model.beta,
                                                k=20)
    return vad_ndcg_at_k,vad_recall_at_k,vad_map_at_k

In [5]:
confidence_model=confidence_expomf.CONFIDENCE_EXPO_MF(n_components=25)
bi_train_data=confidence_model.binarize(raw_train_data)
record_cof=confidence_model.fit_with_confidence(bi_train_data,Raw=raw_train_data,vad_data=vad_data,MAX_ITERATION=15)

model.theta.shape (1013, 25)
model.beta.shape (400, 25)
model.mu.shape (400,)
model.A.shape (1013, 400)
ROUND 0
---updating A---
---updating factors!---
---updating confidence_perItem_U!---
	Validation NDCG@k: 0.01476
	Validation Recall@k: 0.09510
	Validation MAP@k: 0.01555
ROUND 1
---updating A---
---updating factors!---
---updating confidence_perItem_U!---
	Validation NDCG@k: 0.02076
	Validation Recall@k: 0.13232
	Validation MAP@k: 0.02188
ROUND 2
---updating A---
---updating factors!---
---updating confidence_perItem_U!---
	Validation NDCG@k: 0.02359
	Validation Recall@k: 0.15949
	Validation MAP@k: 0.02410
ROUND 3
---updating A---
---updating factors!---
---updating confidence_perItem_U!---
	Validation NDCG@k: 0.02625
	Validation Recall@k: 0.18999
	Validation MAP@k: 0.02620
ROUND 4
---updating A---
---updating factors!---
---updating confidence_perItem_U!---
	Validation NDCG@k: 0.02828
	Validation Recall@k: 0.20763
	Validation MAP@k: 0.02800
ROUND 5
---updating A---
---updating fact

In [6]:
'''More Results are summarized here:

Test NDGC@K 0.075833041714
Test Recall@K 0.240949576877
Test MAP@K 0.0762600971087
'''

'More Results are summarized here:\n\nTest NDGC@K 0.075833041714\nTest Recall@K 0.240949576877\nTest MAP@K 0.0762600971087\n'